In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.lines import Line2D

In [2]:
with open("./data/7o100t100s1p100/feasibility_check.txt", "r") as file:
    lines = file.readlines()
    for line in lines:
        print(line.strip())

every order is in at most one assignment: OK

assignments are never made before information is revealed: OK

bundle picked up before off-time of courier: OK

bundle pickup times respect ready times: OK

dropoffs follow the prescribed sequence:OK

continuity in sequence of origin-destination pairs: OK

departures and arrival time are consistent in time: OK

dropoff times and locations are consistent:OK

pickup times and locations are consistent:OK
FEASIBLE


In [3]:
with open("./data/7o100t100s1p100/solution_performance_Asus_f5d5b10g10x25o1000c0.txt", "r") as file:
    lines = file.readlines()
    for line in lines:
        print(line.strip())

number of orders delivered: 3133 out of 3213
total payment: 32402.50
proportion of couriers receiving minimum guaranteed compensation: 0.13


click-to-door  ready-to-door  ready-to-pickup  click-to-door overage
count        3133.00        3133.00          3133.00                3133.00
mean           38.23          20.82             8.79                   4.76
std            14.62           8.21             5.96                  10.09
min            12.00           5.00             0.00                   0.00
10%            23.00          12.00             3.00                   0.00
50%            36.00          19.00             8.00                   0.00
90%            57.00          31.00            15.00                  17.00
max           169.00          78.00            64.00                 129.00


orders_per_hour  bundles_per_hour  utilization  guaranteed_earnings  order_earnings  payment
count           404.00            404.00       404.00               404.00          40

In [4]:
courier_solution_info = pd.read_csv("./data/7o100t100s1p100/courier_solution_info.txt", sep=' ')
courier_solution_info

,courier_id,timestamp,origin,destination,assignment_id
0,c1,52.0,c1,r12,1
1,c1,100.0,r12,o2594,1
2,c1,113.0,o2594,o941,1
3,c1,127.0,o941,r168,2
4,c1,135.0,r168,o3034,2
...,...,...,...,...,...
6322,c404,711.0,r144,o3156,3154
6323,c404,728.0,o3156,r23,3155
6324,c404,743.0,r23,o583,3155
6325,c404,758.0,o583,r61,3156


In [5]:
df_courier_expand = pd.read_csv("./data/7o100t100s1p100/df_courier_expand.txt", sep=' ')
df_courier_expand

,courier_id,timestamp,x,y
0,c1,0.0,19772.0,927.0
1,c1,1.0,19772.0,927.0
2,c1,2.0,19772.0,927.0
3,c1,3.0,19772.0,927.0
4,c1,4.0,19772.0,927.0
...,...,...,...,...
86857,c404,806.0,24398.0,13068.0
86858,c404,807.0,24398.0,13068.0
86859,c404,808.0,24398.0,13068.0
86860,c404,809.0,24398.0,13068.0


In [6]:
# Read data from the files
restaurants = pd.read_csv("./data/7o100t100s1p100/restaurants.txt", sep='\t')
orders = pd.read_csv("./data/7o100t100s1p100/orders.txt", sep='\t')

# Filter data for courier with id "c1"
courier_c1 = df_courier_expand[df_courier_expand["courier_id"] == "c1"]
#courier_c1 = df_courier_expand[df_courier_expand["courier_id"].isin(["c13", "c9", "c1"])]
courier_c1_solution = courier_solution_info[courier_solution_info["courier_id"] == "c1"]
#courier_c1_solution = courier_solution_info[courier_solution_info["courier_id"].isin(["c13", "c9", "c1"])]
courier_c1_solution = courier_c1_solution[~courier_c1_solution['origin'].str.startswith('c')] # remove first timestamp


# Get visited restaurants and fulfilled orders
visited_restaurants = courier_c1_solution[courier_c1_solution["origin"].str.startswith("r")]
fulfilled_orders = courier_c1_solution[courier_c1_solution["destination"].str.startswith("o")]

restaurant_coords = restaurants.loc[restaurants["restaurant"].isin(visited_restaurants["origin"])]
order_coords = orders.loc[orders["order"].isin(fulfilled_orders["destination"])]
order_coords = order_coords.loc[:, ['order', 'x', 'y']]

In [7]:
# Set up the plot
fig, ax = plt.subplots()
ax.set_title('Courier Movement')
ax.set_xlabel('X')
ax.set_ylabel('Y')

visited_orders = set()

def update(frame):
    ax.clear()
    ax.set_xlim(courier_c1['x'].min() - 100, courier_c1['x'].max() + 100)
    ax.set_ylim(courier_c1['y'].min() - 100, courier_c1['y'].max() + 100)

    # Group courier data by courier_id
    grouped_couriers = courier_c1.groupby('courier_id')

    # Loop through each courier group and plot their positions
    for courier_id, courier_data in grouped_couriers:
        courier = courier_data[courier_data['timestamp'] == frame]

        # Check if there is data for the courier at the current timestamp
        if not courier.empty:
            courier_x = courier['x'].values[0]
            courier_y = courier['y'].values[0]
            ax.scatter(courier_x, courier_y, c='blue', label=f'Courier {courier_id}' if frame == timestamps[0] else '')

    # Plot restaurants
    for index, restaurant in restaurant_coords.iterrows():
        restaurant_color = 'green'
        for _, courier in grouped_couriers:
            courier_at_frame = courier[courier['timestamp'] == frame]
            if not courier_at_frame.empty and courier_at_frame['x'].values[0] == restaurant['x'] and courier_at_frame['y'].values[0] == restaurant['y']:
                restaurant_color = 'yellow'
                break
        ax.scatter(restaurant['x'], restaurant['y'], c=restaurant_color, marker='s', label='Restaurants' if index == 0 else '')

    # Plot orders
    for index, order in order_coords.iterrows():
        visited = False
        for _, courier in grouped_couriers:
            courier_at_frame = courier[courier['timestamp'] == frame]
            if not courier_at_frame.empty and courier_at_frame['x'].values[0] == order['x'] and courier_at_frame['y'].values[0] == order['y']:
                visited_orders.add(index)
                visited = True
                break

        color = 'grey' if index in visited_orders else 'red'
        ax.scatter(order['x'], order['y'], c=color, marker='^', label='Orders' if index == 0 else '')

    # Create custom legend handles and labels
    legend_elements = [
        Line2D([0], [0], marker='o', color='w', label='Courier', markerfacecolor='blue', markersize=8),
        Line2D([0], [0], marker='s', color='w', label='Restaurants', markerfacecolor='green', markersize=8),
        Line2D([0], [0], marker='^', color='w', label='Orders', markerfacecolor='red', markersize=8),
    ]

    ax.legend(handles=legend_elements, loc='best')

timestamps = sorted(courier_c1['timestamp'].unique())
ani = animation.FuncAnimation(fig, update, frames=timestamps, interval=500, repeat=False)

# Save the animation as a GIF file
ani.save('courier_movement.gif', writer='imagemagick', fps=7)

# Close the plot
plt.close()

MovieWriter imagemagick unavailable; using Pillow instead.
